In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import utils
import tensorflow as tf
import json
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from pathlib import Path

# Load Dataset

In [ ]:
ds = utils.create_ds('val', shuffle=True, include_fns=True)
ds_iter = ds.as_numpy_iterator()

In [ ]:
res = ds_iter.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    sample_labels = ', '.join(res[1][0][idx].astype(str).tolist())

    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)

    ax[idx].imshow(sample_image)
    ax[idx].set_title(sample_labels)

# Load Model

In [ ]:
facetracker = load_model('facetracker.h5')

# Predictions DataFrame

In [ ]:
ds_iter = ds.as_numpy_iterator()
images, (classes, coords), fns = ds_iter.next()
yhat_class, yhat_coords = facetracker.predict(images)
dataset_dir = Path(fns[0].decode('utf-8')).parent

In [ ]:
cols = [
    'fn',
    'brendan', 'kara', 
    'pred_brendan', 'pred_kara', 
    'bx1', 'by1', 'bx2', 'by2', 
    'kx1', 'ky1', 'kx2', 'ky2',
    'pred_bx1', 'pred_by1', 'pred_bx2', 'pred_by2', 
    'pred_kx1', 'pred_ky1', 'pred_kx2', 'pred_ky2',
]
df = (pd.DataFrame(np.concatenate((fns[:, np.newaxis], classes, yhat_class, coords, yhat_coords), axis=1), columns=cols)
      .assign(fn = lambda x: x.fn.apply(lambda y: Path(str(y)).stem))
      .set_index('fn')
     )
df

## Add Losses

In [ ]:
for name in ['b', 'k']:
    df.loc[:, f'{name}loss_coord'] = (df[f'{name}x1'] - df[f'pred_{name}x1']).pow(2) + (df[f'{name}y1'] - df[f'pred_{name}y1']).pow(2)
    df.loc[:, f'{name}loss_size'] = (
        ((df[f'pred_{name}x2'] - df[f'pred_{name}x1']) - (df[f'{name}x2'] - df[f'{name}x1'])).pow(2) + 
        ((df[f'pred_{name}y2'] - df[f'pred_{name}y1']) - (df[f'{name}y2'] - df[f'{name}y1'])).pow(2)
    )
    df.loc[:, f'{name}loss'] = df.loc[:, f'{name}loss_coord'] + df.loc[:, f'{name}loss_size']

df

# Test Showing First Four Images

In [ ]:
def points_percent_to_pixel(x, y, pixel_size=120):
    x = int(x * pixel_size)
    y = int(y * pixel_size)
    return (x, y)

In [ ]:
name = 'brendan'
letter = name[0]
df_name = df.loc[df.loc[:, name] == 1, :].sort_values('kloss', ascending=True)
df_name.head(4)[['kx1', 'ky1', 'kx2', 'ky2','pred_kx1', 'pred_ky1', 'pred_kx2', 'pred_ky2',]]

four_iter = iter(range(0, len(df_name), 4))

In [ ]:
i = next(four_iter)
df_errors = df_name.iloc[i:i+4]
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, (img_fn, row) in enumerate(df_errors.iterrows()):
    image = utils.load_image(str(dataset_dir / f'{img_fn}.jpg'))
    image = tf.image.resize(image, (120,120)).numpy()
    # actual
    start_point = points_percent_to_pixel(row[f'{letter}x1'], row[f'{letter}y1'])
    end_point = points_percent_to_pixel(row[f'{letter}x2'], row[f'{letter}y2'])
    cv2.rectangle(image, start_point, end_point, (9, 121, 105), 2)

    # predicted
    start_point = points_percent_to_pixel(row[f'pred_{letter}x1'], row[f'pred_{letter}y1'])
    end_point = points_percent_to_pixel(row[f'pred_{letter}x2'], row[f'pred_{letter}y2'])
    cv2.rectangle(image, start_point, end_point, (236, 255, 220), 1)

    ax[idx].imshow(image.astype(int))
    # ax[idx].set_title('')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
(df.loc[:, 'pred_brendan'] > threshold).astype(int).value_counts()

In [ ]:
df.loc[:, 'brendan'].value_counts()

In [ ]:
name = 'brendan'
threshold = 0.9
confusion_matrix(df.loc[:, 'brendan'].astype('int64'),
                 (df.loc[:, 'pred_brendan'] > threshold).astype('int64'))

In [ ]:
name = 'kara'
threshold = 0.9
confusion_matrix(df.loc[:, name].astype('int64'),
                 (df.loc[:, f'pred_{name}'] > threshold).astype('int64'))